In [1]:
import pandas as pd

from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
data = pd.read_csv('aivleschool_qa_updated.csv', encoding='utf-8')
display(data)

,구분,QA
0,모집/선발,최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제...
1,모집/선발,35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교...
2,모집/선발,미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함...
3,모집/선발,"직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시..."
4,모집/선발,아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 1...
...,...,...
63,기타,AICE (기존 AIFB) 자격증이 있어야 유리한가요?\n네. AICE (기존 AI...
64,기타,AICE 시험관련 문의는 어디에 해야하나요?\nhelp@aice.study 으로 문...
65,기타,국민취업지원제도를 신청하고 싶습니다.\n국민취업지원제도 신청과 관련 사항은 거주지 ...
66,기타,국민취업지원제도를 안해도 교육 참여되나요?\n국민취업제도를 신청하지 않아도 KT 에...


In [3]:
'''
Vector DB
Embedding 모델: text-embedding-ada-002
DB 경로: ./db
'''
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
database = Chroma(persist_directory='./database', embedding_function=embeddings)

In [4]:
'''
DF to Vector DB
Documents 생성
DB Insert
'''
# Documents 생성
qa_categories = data['구분'].tolist()
qa_contents = data['QA'].tolist()
qa_list = []
for pair in zip(qa_categories, qa_contents):
    qa_list.append(pair)

documents = []
for qa in qa_list:
    qa_category = dict()
    qa_category['category'] = qa[0]
    qa_content = qa[1]
    document = Document(page_content=qa_content, metadata=qa_category)
    documents.append(document)

documents

[Document(page_content='최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.', metadata={'category': '모집/선발'}),
 Document(page_content='35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교육입니다. 단, 모집시점에 35세라도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다. (예: 5기 지원 기준 1989년 1월 1일 이후 출생자)', metadata={'category': '모집/선발'}),
 Document(page_content='미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함이 있는 근로인 경우,\n2) 고용보험에 미가입한 경우,\n3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우\n단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.', metadata={'category': '모집/선발'}),
 Document(page_content='직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다. 향후 취업여부를 확인할 예정입니다.', metadata={'category': '모집/선발'}),
 Document(page_content='아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 15시간/주 미만 근로인 경우에만 미취업자로 간주하여 지원이 가능합니다.\n고용보험에 가입되어 있지 않는 경우에는 특수형태근로자/고용보험미가입근로자로 분류되어 근로시간과 훈련시간이 중복되지 않는다면 훈련 수강은 가능하나, 훈련장려금이 지급되지 않습니다.\n다만, 특수

In [5]:
# DB Insert
database.add_documents(documents)

['1558f83d-a086-4a3a-b0b7-78c123b95d28',
 '8c2ada72-78a3-4539-b47f-435b47a7ee23',
 '3633126b-f4f9-40c9-8b99-ef566ea1aa19',
 '35dee1cf-6379-4001-93ee-4fe5fdbe9b55',
 'aed83d05-dfeb-4608-82df-6d1f3c426d2d',
 'd166919f-4e0f-4d09-b136-ab85a24db932',
 '377067fe-aae3-481c-8005-598107e8b068',
 'ed4bd991-2db3-49d0-81e3-8eb9c13030c8',
 'db9cc4f3-99a3-4d34-81c4-b30c974f6b02',
 '98d5284c-494d-4454-9c0a-0222eb736cba',
 'ee64b7b1-8986-48e4-b8b9-3814099a7793',
 'bc5e9920-177a-4091-b2f1-324e16fe15de',
 'cafd92d3-838d-4f8b-bae0-2a0fedafb9f9',
 'e66be409-88e6-4d01-9e77-d95c3e363c43',
 '6e2a0274-92c3-4c82-8fb2-53376b6b8ed7',
 '07b72cbf-9e02-491e-9f5f-a3d19ece3fb4',
 'cb108639-76b6-4194-aa5f-1b5368910e12',
 '68048273-d1df-42d9-95ba-6988e95a6431',
 '92636cf5-f2d4-4a10-8796-7cb240ff619f',
 '0be7cabd-de2c-49fe-9bd3-e07e3ea31b20',
 'b19a2b6b-1664-4822-b205-b5abe7060da5',
 '86237e5a-f125-4404-9719-5cf8e9e95a77',
 '5042dde5-dfec-4d08-8557-4063c892bbff',
 '6ed4382d-265f-4794-96b4-b6c5ceca8af6',
 'fed858c1-891e-

In [6]:
database.get()

{'ids': ['030d8314-7f3b-44a7-a538-64b3cd88fe88',
  '07b72cbf-9e02-491e-9f5f-a3d19ece3fb4',
  '0be7cabd-de2c-49fe-9bd3-e07e3ea31b20',
  '0d14437d-2109-4c49-89a7-993d31695ec2',
  '1558f83d-a086-4a3a-b0b7-78c123b95d28',
  '1f2b712b-1fdd-4009-869b-8fca4d018dc3',
  '24cb1cbd-7394-4cd8-9e7a-c34ed9146e8f',
  '2f34e19a-3042-47a2-8587-94ee28e34d2d',
  '32064188-6bf6-4639-b151-f48794b29120',
  '322d08b1-5e6f-4c7d-8aee-de989c78ae99',
  '35dee1cf-6379-4001-93ee-4fe5fdbe9b55',
  '3633126b-f4f9-40c9-8b99-ef566ea1aa19',
  '377067fe-aae3-481c-8005-598107e8b068',
  '3dc50de6-7dd6-4390-9661-440ba30f2df3',
  '3f5db448-dc5c-46a9-a49a-7500ea1d1365',
  '3fbc373f-6720-41c9-b861-387468d9f423',
  '40e3ea88-95ec-4143-89ab-2c6b6da3c041',
  '5042dde5-dfec-4d08-8557-4063c892bbff',
  '5181ee1b-f843-40c0-a4c3-0870474c4811',
  '574f9f5f-ef0d-45af-a324-5cdb857972d0',
  '66d1c11c-601d-4837-b1c3-a3d4f518e6d5',
  '68048273-d1df-42d9-95ba-6988e95a6431',
  '6949862f-ec81-4a70-a682-c4a4c3b175d5',
  '69e6b594-7702-4419-bb48-